In [9]:
import numpy as np
import pandas as pd

# Linear Regression


Question: Write code to implement linear regression from scratch without using
any machine learning libraries like scikit-learn.

Answer Guidance: Be prepared to:
    *	Handle data input and preprocessing.
	*	Compute the OLS estimates manually.
	*	Implement matrix operations using libraries like NumPy.
	*	Optionally, write functions to predict new data points and calculate metrics like Mean Squared Error (MSE).


In [10]:
df = pd.read_csv('data/df_regression.csv') 
X = df[['X1','X2','X3']]
y = df['y']


In [11]:
# Generalizing for Multiple Linear Regression

def multiple_linear_regression(X,y):

    """
    Estimates the coefficients for a multiple linear regression model using the normal equation.

    This function computes the coefficients (weights) for a multiple linear regression model
    by solving the normal equation: beta = (X^T * X)^(-1) * X^T * y. It works for any number
    of predictors (independent variables) and includes an intercept term.

    Parameters:
    ----------
    X : array-like or list of lists
        Feature matrix containing the independent variables.
        Shape: (n_samples, n_features)
    
    y : array-like or list
        Target vector containing the dependent variable.
        Shape: (n_samples,)

    Returns:
    -------
    beta : numpy.ndarray
        Coefficient vector including the intercept term and coefficients for each predictor.
        Shape: (n_features + 1,)
    """    
    
    # Conver inout to numpy (in case they are dataframes..)
    X = np.array(X)
    y = np.array(y)

    # Add intercept
    ones = np.ones((X.shape[0], 1))  # vector of ones
    X = np.concatenate((ones, X), axis =1)

    #Compute the elements of the normal equations
    #     A = (X'X)**(-1) 
    #     B = X'Y
    #  beta = A* B is then the coefficient vector

    A = np.linalg.inv(( X.T.dot(X)))
    B = X.T.dot(y)

    beta = A.dot(B)

    return(beta)


multiple_linear_regression(X,y)


array([9.19994425, 3.08318585, 1.52085497, 2.13173927])

Note: In NumPy, the * operator performs element-wise multiplication, not matrix multiplication.
To perform matrix multiplication, use the .dot() method or the @ operator.


`A.dot(B)`:  Correct way to perform matrix multiplication in NumPy

In [12]:
def predict(X_new, beta):
    
    # Convert input to numpy array
    X_new = np.array(X_new)

    # Add intercept term
    ones = np.ones((X_new.shape[0]),1 )
    X_new = np.concat((ones, X_new), axis = 1)

    # Calculate prediction 

    y_pred = X_new.dot(beta)

    return y_pred

def mean_squared_error(y_true, y_pred):
    mse = np.mean((y_true - y_pred)**2)
    return mse


A possible follow up question would be **How would you implement linear 
regression using gradient descent instead of the normal equation?** 

This gradient approach is especially useful when dealing with a large number of 
features or when the matrix inversion in the normal equation becomes computationally 
expensive. Gradient descent works by iteratively adjusting the coefficients to minimize the cost function, typically the Mean Squared Error (MSE) for linear regression.

### Key Concepts of Gradient Descent for Linear Regression

#### 3. Cost Function
The cost function measures how well the model's predictions match the actual data. For linear regression, the cost function is the Mean Squared Error (MSE):

$$
J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)} \right)^2
$$

where:
- $ m $ is the number of training examples.
- $ h_{\theta}(x^{(i)}) $ is the predicted value for the $ i $-th sample.
- $ y^{(i)} $ is the actual value for the $ i $-th sample.

The goal of gradient descent is to find the values of $ \theta $ that minimize $ J(\theta) \).

#### 4. Gradient Descent Algorithm
Gradient Descent is an iterative optimization algorithm that updates the coefficients in the direction of the negative gradient of the cost function:

$$
\theta_j = \theta_j - \alpha \frac{\partial J(\theta)}{\partial \theta_j}
$$

where:
- $ \theta_j $ is the $ j $-th parameter.
- $ \alpha $ is the learning rate, controlling the step size.
- $ \frac{\partial J(\theta)}{\partial \theta_j} $ is the partial derivative of the cost function with respect to $ \theta_j $.

#### 5. Gradient Calculation
For linear regression, the gradient of the cost function with respect to each parameter $ \theta_j $ is calculated as follows:

$$
\frac{\partial J(\theta)}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)} \right) x_j^{(i)}
$$

where:
- $ x_j^{(i)} $ is the value of the $ j $-th feature for the $ i $-th sample.

#### 6. Update Rule
The update rule for each coefficient in the gradient descent algorithm is:

$$
\theta_j = \theta_j - \alpha \frac{1}{m} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)} \right) x_j^{(i)}
$$

This update rule is applied iteratively for all coefficients until convergence.

#### 7. Learning Rate
The learning rate $ \alpha $ controls the size of the steps taken towards the minimum of the cost function. Choosing the right learning rate is crucial:
- **Too small:** The algorithm will take too long to converge.
- **Too large:** The algorithm may overshoot the minimum or even diverge.

#### 8. Convergence
The gradient descent algorithm converges when the cost function $ J(\theta) $ reaches its minimum value. This can be monitored by checking the change in cost over iterations or by setting a maximum number of iterations.

#### 9. Variants of Gradient Descent
- **Batch Gradient Descent:** Uses all training examples to compute the gradient.
- **Stochastic Gradient Descent (SGD):** Uses one training example at a time to compute the gradient, which can be faster but introduces more noise in the updates.
- **Mini-batch Gradient Descent:** Uses a small subset (mini-batch) of the training examples to compute the gradient, balancing the efficiency and stability of the updates.

#### 10. Summary
Gradient Descent is a powerful algorithm for optimizing linear regression models, especially when dealing with large datasets. By iteratively updating the coefficients in the direction of the negative gradient of the cost function, it finds the optimal parameters that minimize the error between the predicted and actual values.

In [15]:
# So let's write these beast

def linear_regression_gradient_descent(X, y, alpha=0.01, n_iterations=1000):
    # alpha is the learning rate`
    # Convert input to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # Number of training examples and features
    m = X.shape[0]  # Number of samples
    n = X.shape[1]  # Number of features

    # Add intercept
    ones = np.ones((m, 1))  # vector of ones
    X = np.concatenate((ones, X), axis =1)

    # Initialize theta (coefficient vector) with zeros
    beta = np.zeros(n+1)

    # Initialize cost function (optional)
    cost_history = []

    ###########################################################################
    # Gradient Descent

    for iter in range(n_iterations):

        y_pred = X.dot(beta)               # Calculate predictions
        error = y_pred - y                  # Calculate errors
        gradients = (1/m) *  np.dot(X.T, error) # Calculate gradients
        beta = beta - alpha * gradients   # Update rule

        cost_history.append( (1/(2*m)) * np.sum(error**2)) # Optional: update cost_history 

    return beta, cost_history

linear_regression_gradient_descent(X,y)

(array([5.67238675, 3.26813627, 1.63272398, 2.60832671]),
 [np.float64(1063.5530721022635),
  np.float64(380.5420589777542),
  np.float64(152.31498412972596),
  np.float64(72.75486483283463),
  np.float64(42.618463147511385),
  np.float64(29.521198359447112),
  np.float64(22.73980593416489),
  np.float64(18.608818921261634),
  np.float64(15.79191578454744),
  np.float64(13.743207205703841),
  np.float64(12.20092152785298),
  np.float64(11.016655709441636),
  np.float64(10.094764250172247),
  np.float64(9.368686325135352),
  np.float64(8.790239051909047),
  np.float64(8.323876102542217),
  np.float64(7.943131149684714),
  np.float64(7.62820049126274),
  np.float64(7.364216757926175),
  np.float64(7.139986627613239),
  np.float64(6.947057820037653),
  np.float64(6.779026115515249),
  np.float64(6.631019492241544),
  np.float64(6.499313677520949),
  np.float64(6.381045439004445),
  np.float64(6.273998651046979),
  np.float64(6.176444577226952),
  np.float64(6.087022555597802),
  np.float6

# Logistic Regression.

The logistic regression model arises from the desire to model the posterior 
probabilities of the K classes via linear functions in x, while at the same time
ensuring that they sum to one and remain in [0,1].

The **sigmoid function** is maps any real-valued number into a value between 0 
and 1, making it suitable for modeling probabilities:

$$   \sigma(z) = \frac{1}{1+ e^(-z)}  $$

where $ z = \beta_0 + \beta X$




In [ ]:


def logistic_regression(X, y, alpha=0.01, n_iterations=1000):
    # alpha is the learning rate`
    # Convert input to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # Number of training examples and features
    m = X.shape[0]  # Number of samples
    n = X.shape[1]  # Number of features

    # Add intercept
    ones = np.ones((m, 1))  # vector of ones
    X = np.concatenate((ones, X), axis =1)

    # Initialize theta (coefficient vector) with zeros
    beta = np.zeros(n+1)

    ###########################################################################
    # Gradient Descent
    ###########################################################################

    for iter in range(n_iterations):
        
        z = X.dot(beta)
        y_pred = 1/(1 + np.exp(-z))              # Calculate predictions
        error = y_pred - y                  # Calculate errors
        gradients = (1/m) *  np.dot(X.T,error) # Calculate gradients
        beta = beta - alpha * gradients   # Update rule

    return beta

linear_regression_gradient_descent(X,y)

In [17]:

def regressions(X, y, type = 'linear', alpha=0.01, n_iterations=1000):

    """
    The implementations based on gradient descent for linear and logistic 
    regression are almost identical. The only difference is the computation of 
    y_pred. This code makes this clear, generalizing the linear regression 
    function to include a new parameter, 'type':

    Parameters:
    ----------
    X : array-like
        Feature matrix, shape (n_samples, n_features).
    
    y : array-like
        Target vector, shape (n_samples,).
    
    type : string, optional
        Either 'linear' or 'logistic'. Defines the type of regression performed.
        Default is 'linear'.
    
    alpha : float, optional
        Learning rate for gradient descent. Default is 0.01.
    
    n_iterations : int, optional
        Number of iterations for gradient descent. Default is 1000.

    Returns:
    -------
    beta : numpy.ndarray
        Coefficient vector, shape (n_features + 1,).
    """

    # alpha is the learning rate`
    # Convert input to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # Number of training examples and features
    m = X.shape[0]  # Number of samples
    n = X.shape[1]  # Number of features

    # Add intercept
    ones = np.ones((m, 1))  # vector of ones
    X = np.concatenate((ones, X), axis =1)

    # Initialize beta (coefficient vector) with zeros
    beta = np.zeros(n+1)

    ###########################################################################
    # Gradient Descent
    ###########################################################################

    for iter in range(n_iterations):
        
        if type == 'linear':
            y_pred = X.dot(beta)  
        elif type == 'logistic':
            z = X.dot(beta)
            y_pred = 1/(1 + np.exp(-z))
        else:
            raise ValueError('Type must be either linear or logistic') 
        # Calculate predictions
        error = y_pred - y                  # Calculate errors
        gradients = (1/m) *  np.dot(X.T,error) # Calculate gradients
        beta = beta - alpha * gradients   # Update rule

    return beta




In [ ]:
m = X.shape[0]
n = X.shape[]

### Additional notes on logistic regression:


Question: How would you modify your logistic regression implementation to 
include L2 regularization?

We can add a regularization term to the cost function to penalize large weights.

    - write deep-dive.

Question: Which metrics would you use to evaluate your logistic regression model,
and how would you implement them?

Discus accuracy metrics:
	•	Precision
	•	Recall
	•	F1 Score
	•	Confusion Matrix
	•	ROC Curve and AUC

To-do: write code

Question: Is feature scaling necessary for logistic regression? Implement 
feature scaling in your code if needed.

while logistic regression doesn’t require feature scaling for correctness, 
scaling can improve the convergence rate of gradient descent. Show how to 
standardize features using z-score normalization.

What are the key assumptions of logistic regression, and how can you test them 
with your data?

List assumptions such as:
	•	Linearity of independent variables and log-odds.
	•	Independence of errors.
	•	Lack of multicollinearity.
	•	Large sample size.

    